In [6]:
import os
import torch
import pandas as pd
import pdfplumber
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, TrainingArguments
from datasets import Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

C:\Users\OMEN\AppData\Local\Temp\ipykernel_23888\2067075365.py:5: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [1]:
!pip install unsloth PyMuPDF pdfplumber accelerate bitsandbytes torch transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Instal PyTorch yang kompatibel dengan CUDA
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Instal Unsloth (khusus untuk Conda/Windows) dan library lainnya
!pip install "unsloth[conda]>=2024.6"
!pip install PyMuPDF pdfplumber pandas datasets "trl<0.9.0" peft accelerate

Looking in indexes: https://download.pytorch.org/whl/cu121, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
    --------------------------------------- 3.7/212.5 MB 27.3 MB/s eta 0:00:08
   -- ------------------------------------- 11.0/212.5 MB 29.9 MB/s eta 0:00:07
   --- ------------------------------------ 17.8/212.5 MB 31.2 MB/s eta 0:00:07
   ---- ----------------------------------- 24.9/212.5 MB 32.2 MB/s eta 0:00:06
   ----- ---------------------------------- 31.7/212.5 MB 32.5 MB/s eta 0:00:06
   ------- -------------------------------- 39.1/212.5 MB 33.1 MB/s eta 0:00:06
   -------- ------------------------------- 45.6/212.5 MB 33.0 MB/s eta 0:00:06
   --------- ------------------------------ 52.4/212.5 MB 33.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.7.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Attempting uninstall: trl
    Found existing installation: trl 0.19.0
    Uninstalling trl-0.19.0:
      Successfully uninstalled trl-0.19.0


In [ ]:
# ===================================================================
# 1. IMPORT LIBRARY (DENGAN TAMBAHAN)
# ===================================================================
import os
import torch
import pandas as pd
import pdfplumber
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, TrainingArguments
from datasets import Dataset
from trl import SFTTrainer
from huggingface_hub import notebook_login

# ===================================================================
# 2. LOGIN KE HUGGING FACE HUB
# ===================================================================
print("Silakan login ke akun Hugging Face Anda...")
notebook_login()

# ===================================================================
# 3. FUNGSI UNTUK MEMPROSES DATASET DARI FOLDER
# ===================================================================
def create_dataset_from_folders(pdf_folder, summary_folder):
    """
    Membaca pasangan file PDF dan TXT dari folder, mengekstrak isinya,
    dan mengubahnya menjadi Hugging Face Dataset.
    """
    data = []
    print(f"Mulai memproses data dari folder: {pdf_folder}")

    for pdf_filename in os.listdir(pdf_folder):
        if pdf_filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, pdf_filename)
            
            summary_filename = pdf_filename.replace('.pdf', '.txt')
            summary_path = os.path.join(summary_folder, summary_filename)

            if os.path.exists(summary_path):
                pdf_text = ""
                try:
                    with pdfplumber.open(pdf_path) as pdf:
                        for page in pdf.pages:
                            text = page.extract_text()
                            if text:
                                pdf_text += text + "\n"
                except Exception as e:
                    print(f"Gagal membaca {pdf_filename}: {e}")
                    continue
                
                with open(summary_path, 'r', encoding='utf-8') as f:
                    summary_text = f.read()
                
                data.append({"input_text": pdf_text, "summary": summary_text})
            else:
                print(f"Peringatan: File ringkasan {summary_filename} tidak ditemukan.")

    # PERBAIKAN: Cek apakah ada data sebelum membuat DataFrame
    if not data:
        raise ValueError("Tidak ada pasangan data (PDF dan TXT) yang berhasil ditemukan. Pastikan nama file cocok dan folder tidak kosong.")
        
    df = pd.DataFrame(data)
    dataset = Dataset.from_pandas(df)
    
    print(f"✅ Dataset berhasil dibuat dengan {len(dataset)} contoh.")
    return dataset

# ===================================================================
# 4. FUNGSI UNTUK MEMFORMAT PROMPT
# ===================================================================
prompt_template = """
[INST] Anda adalah seorang analis keuangan. Berdasarkan laporan keuangan berikut, buatlah ringkasan terstruktur yang mencakup Laporan Posisi Keuangan, Laporan Laba Rugi, dan Laporan Arus Kas.

### Laporan Keuangan:
{input_text}

[/INST]
### Ringkasan Analisis:
{summary}
"""

def format_dataset(examples):
    formatted_texts = []
    for input_text, summary in zip(examples["input_text"], examples["summary"]):
        formatted_text = prompt_template.format(input_text=input_text, summary=summary) + tokenizer.eos_token
        formatted_texts.append(formatted_text)
    return { "text": formatted_texts }

# ===================================================================
# 5. KONFIGURASI DAN FINE-TUNING
# ===================================================================

# --- GANTI PATH INI SESUAI LOKASI DATA ANDA ---
pdf_folder_path = "Data/File"
summary_folder_path = "Data/Summarize"
hub_model_name = "FinMA-7B-Financial-Analysis-v1" 

# ----- Muat dan siapkan dataset -----
try:
    raw_dataset = create_dataset_from_folders(pdf_folder_path, summary_folder_path)

    # ----- Muat Model dan Tokenizer dengan Unsloth -----
    model_name = "TheFinAI/finma-7b-nlp"
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=4096,
        dtype=None,
        load_in_4bit=True,
    )

    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": "<pad>"})
        model.resize_token_embeddings(len(tokenizer))

    # ----- Format dataset -----
    formatted_dataset = raw_dataset.map(format_dataset, batched=True, remove_columns=raw_dataset.column_names)

    # ----- Konfigurasi LoRA -----
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_dropout=0.05,
        bias="none",
        use_gradient_checkpointing=True,
        random_state=42,
    )

    # ----- Konfigurasi Trainer -----
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=formatted_dataset,
        dataset_text_field="text",
        max_seq_length=4096,
        dataset_num_proc=2,
        args=TrainingArguments(
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=10,
            num_train_epochs=1,
            learning_rate=2e-5,
            fp16=not torch.cuda.is_bf16_supported(),
            bf16=torch.cuda.is_bf16_supported(),
            logging_steps=1,
            optim="adamw_8bit",
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=42,
            output_dir="outputs",
        ),
    )

    # ----- Mulai Fine-tuning -----
    print("🚀 Memulai proses fine-tuning...")
    trainer.train()
    print("✅ Fine-tuning selesai!")

    # ===================================================================
    # 6. SIMPAN DAN UNGGAH MODEL KE HUGGING FACE HUB
    # ===================================================================
    print("Menyimpan model ke direktori 'outputs'...")
    trainer.save_model("outputs")

    print(f"Mengunggah model ke Hugging Face Hub dengan nama: {hub_model_name}")
    model.push_to_hub(hub_model_name, use_auth_token=True)
    tokenizer.push_to_hub(hub_model_name, use_auth_token=True)

    print("🎉 Model Anda berhasil diunggah dan siap untuk dideploy!")

except ValueError as e:
    print(f"\n❌ PROSES DIHENTIKAN: {e}")

NotImplementedError: Unsloth currently only works on NVIDIA GPUs and Intel GPUs.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TheFinAI/finma-7b-nlp")
model = AutoModelForCausalLM.from_pretrained("TheFinAI/finma-7b-nlp")

In [ ]:
# ===================================================================
# 1. INSTALASI LIBRARY YANG DIPERLUKAN (Lebih Ringan)
# ===================================================================
!pip install "unsloth[kaggle-new]>=2024.6" --upgrade
!pip install PyMuPDF pdfplumber

# ===================================================================
# 2. IMPORT LIBRARY
# ===================================================================
import torch
import pdfplumber
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import os
import re
from typing import List, Dict

# ===================================================================
# 3. PEMUATAN MODEL FinMA DENGAN UNSLOTH
# ===================================================================
model_name = "TheFinAI/finma-7b-nlp"
max_seq_length = 2048 
dtype = None 
load_in_4bit = True

print(f"Memuat model: {model_name}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("✅ Model FinMA berhasil dimuat dengan Unsloth.")

# =============================================================================
# 4. FUNGSI PEMROSESAN PDF YANG DISEREDERHANAKAN
# =============================================================================
def extract_relevant_financial_text(pdf_path: str) -> str:
    """
    Fungsi ini membuka PDF, mengekstrak teks dari setiap halaman, dan
    menggabungkan semua teks menjadi satu konteks besar untuk dianalisis.
    """
    if not os.path.exists(pdf_path):
        print(f"❌ File tidak ditemukan: {pdf_path}")
        return None
        
    full_text = ""
    print(f"Memproses file: {os.path.basename(pdf_path)}...")
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                page_text = page.extract_text(x_tolerance=2)
                if page_text:
                    # Menambahkan penanda halaman dan membersihkan teks
                    cleaned_text = re.sub(r'\s{2,}', ' ', page_text)
                    full_text += f"\n\n--- HALAMAN {i+1} ---\n{cleaned_text}"
    except Exception as e:
        print(f"Error saat memproses PDF: {e}")
        return None

    print(f"✅ Berhasil mengekstrak teks dari PDF.")
    return full_text

# ===================================================================
# 5. FUNGSI ANALISIS MENGGUNAKAN PROMPT [INST]
# ===================================================================
def get_financial_analysis(konteks):
    """Menghasilkan analisis terstruktur menggunakan model FinMA."""
    prompt_template = """
[INST] Anda adalah seorang analis keuangan ahli dari Indonesia. Berdasarkan Konteks laporan keuangan berikut, berikan analisis keuangan yang komprehensif. Fokus pada tiga area utama:

1.  **Analisis Laporan Posisi Keuangan (Neraca):** Evaluasi kesehatan keuangan perusahaan berdasarkan aset, liabilitas, dan ekuitasnya.
2.  **Analisis Laporan Laba Rugi:** Analisis profitabilitas berdasarkan pendapatan dan beban.
3.  **Analisis Laporan Arus Kas:** Analisis bagaimana perusahaan menghasilkan dan menggunakan kas.

### Konteks:
{konteks_baru}
[/INST]
### Respons:
"""
    
    final_prompt = prompt_template.format(konteks_baru=konteks)
    inputs = tokenizer([final_prompt], return_tensors="pt").to("cuda")
    
    torch.cuda.empty_cache()
    
    outputs = model.generate(
        **inputs, 
        max_new_tokens=1024,
        use_cache=True,
        do_sample=True,
        temperature=0.1,
        top_p=0.9
    )
    
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer_part = response_text.split("[/INST]")[-1].strip().replace("### Respons:", "").strip()
    return answer_part

# ===================================================================
# 6. EKSEKUSI LANGSUNG PADA FILE PDF TERTENTU
# ===================================================================
# --- Ganti path file PDF di sini ---
pdf_path_to_analyze = "/kaggle/input/datathon/FinancialStatement-2025-I-AADI.pdf"

if os.path.exists(pdf_path_to_analyze):
    print(f"\nMemproses file: {pdf_path_to_analyze}...")
    
    # Langkah 1: Ekstrak seluruh teks yang relevan dari PDF
    document_context = extract_relevant_financial_text(pdf_path_to_analyze)
    
    if document_context:
        print("\nSedang menghasilkan analisis dari model FinMA...")
        # Langkah 2: Dapatkan analisis dari model
        analysis_result = get_financial_analysis(document_context)
        
        # Langkah 3: Tampilkan hasil
        print("\n" + "="*25 + " HASIL ANALISIS KEUANGAN " + "="*25)
        print(f"Untuk Dokumen: {os.path.basename(pdf_path_to_analyze)}\n")
        print(analysis_result)
        print("="*77 + "\n")
    else:
        print("Gagal melanjutkan karena tidak ada teks yang bisa diproses dari PDF.")
else:
    print(f"❌ File tidak ditemukan: {pdf_path_to_analyze}")
    print("Pastikan file PDF berada di direktori yang benar.")